## @Mohammed Ombadi (ombadi@lbl.gov) _ Nature 2023, Ombadi et al. "A warming-induced reduction in snow fraction amplifies rainfall extremes"

## Import Libraries

In [5]:
# Basic libraries
import os
import glob
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Other
import netCDF4 as nc

 


from sklearn.linear_model import LinearRegression
from datetime import datetime

import scipy as sc

import math
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn
from ridgeplot import ridgeplot
import matplotlib.patheffects as mpe

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## 1: Compute average global surface temperatures from 2015 to 2100 for each model and scenario:

In [30]:
models = ['AWI', 'BCC', 'CMCC', 'EC-Earth3', 'GFDL', 'MPI-ESM1-2-HR', 'TaiESM1'] #List of models
scenario = ['ssp126', 'ssp245', 'ssp370', 'ssp585'] #List of scenarios

# Iterate over scenarios
for s in range(len(scenario)):
    
    # Pre-allocate a dataframe for the results with size (n*m)
    # where:
    # n = years (2015 - 2100)
    # m = models
    surface_temp = pd.DataFrame(index = list(np.arange(2015, 2101)), columns= models)
    month_idx = np.arange(1, (86*12), 12)
    
    # Iterate over models
    for m in range(len(models)):
        
        path = './Raw data/CMIP6_sample_data/Surface Temperature Data/'
        files1 = glob.glob(path + '*/*/*_' + models[m] + '*' + scenario[s] + '*.nc', recursive= True) 
        files2 = glob.glob(path + '*/*' + models[m] + '*' + scenario[s] + '*.nc', recursive= True)
        files = files1 + files2
        
        if (len(files) >= 1):
            
            # Pre-allocation
            ds = nc.Dataset(files[0])
            d = ds['ts'][:]
            data = np.empty([d.shape[1], d.shape[2], 1])
            
            for f in range(len(files)):
                
                ds = nc.Dataset(files[f])
                d = ds['ts'][:]
                d = np.transpose(d, (1, 2, 0))
                data = np.concatenate((data, d), axis=2) # size (lat*lon*months from Jan, 2015 to Dec, 2100)
                
            data = data[:,:,1:] #remove the pre-allocation layer
            
            for i in range(surface_temp.shape[0]):
                
                surface_temp[models[m]].iloc[i] = np.nanmean(np.nanmean(
                    np.nanmean(data[:,:,month_idx[i]:(month_idx[i]+12)])))

# This is only an example on one model (MPI-ESM1-2-HR)
# To reproduce the results, all input data need to be downloaded in the model. Currently, the directory only have data
# for model MPI-ESM1-2-HR

print(surface_temp.head()) 

# To access the results of this process for all models, see "./Results/Warming levels_By scenario_model_year/"
                

      AWI  BCC CMCC EC-Earth3 GFDL MPI-ESM1-2-HR TaiESM1
2015  NaN  NaN  NaN       NaN  NaN    279.669135     NaN
2016  NaN  NaN  NaN       NaN  NaN    279.790336     NaN
2017  NaN  NaN  NaN       NaN  NaN    279.934452     NaN
2018  NaN  NaN  NaN       NaN  NaN    279.799637     NaN
2019  NaN  NaN  NaN       NaN  NaN    280.299843     NaN


## 2: Use the estimates computed from (1) to calculate a moving average for the temperature change relative to the reference period:

In [63]:
# First, the vector below (T_ref) shows the estimates of global average temperature for the 7 models: 
# ('AWI', 'BCC', 'CMCC', 'EC-Earth3', 'GFDL', 'MPI-ESM1-2-HR', 'TaiESM1')
# These estimates are from the reference period (1950 - 1979) in units of Kelvin

T_ref = np.array([278.4688, 278.7968, 279.0411, 279.5317, 278.2650, 278.8195, 277.3771])

files = glob.glob('./Results/Warming levels_By scenario_model_year/*.csv') 
                                                                           
# Iterate over scenarios since each file has estimates of all models for one scenario
for f in range(len(files)):
    
    data = pd.read_csv(files[f])
    #data = data.rename(columns={"Unnamed: 0": "year"})
    
    
    # Pre-allocate a dataframe for the results with size = n * (m+2),
    # where:
    # n = number of moving windows (2015-2044, 2016-2045 .... 2071-2100)
    # Columns are m models + two columns for begin and end year
    
    df = pd.DataFrame(index = list(range(data.shape[0]-29)), columns= data.columns[1:])
    df['begin_year'] = data['year'].iloc[:-29].values
    df['end_year'] = data['year'].iloc[29:].values
    
    # Compute 30-years moving average starting from (2015-2044) and up to (2071-2100) and substract T_ref
    for i in range(data.shape[0]-29):
        df.iloc[i,:-2] = np.nanmean(data.iloc[i:(i+30), 1:], axis= 0) - T_ref
    
    
    print('**********_' + os.path.basename(files[f])[12:18])
    #print(df.head())
    
 ### ----------------------------------------------------------------------------------------------------------- ###
     ## 3: Use the estimates in "df" to identify the years for each model and scenario that falls within 
      ##                                 (+1.5 K, +2 k, +3 k and +4 k)
    
    # Find the optimum years that give warming levels of 1.5, 2, 3 and 4 degrees
    warming_levels = [1.5, 2, 3, 4]
    
    for w in range(len(warming_levels)):
        
        print('warming level = ' + str(warming_levels[w]))
        dummy = pd.DataFrame().reindex_like(df)
        dummy.iloc[:,1:] = abs(df.iloc[:,1:] - warming_levels[w])
        dummy['year'] = df['begin_year']
        dummy = dummy.astype('float')
        
        for j in np.arange(1, dummy.shape[1]):
            
            x = np.nanmin(dummy.iloc[:, j])
            index = dummy[[dummy.columns[j]]].idxmin()
            
            if (x < 0.1):
                
                print(dummy.columns[j] + '  ,epsilon = ' + str(x) + ', years start at= ' 
                      + str(dummy['year'][index]) )
                
        print('--------------------------------------------')

**********_ssp126
warming level = 1.5
BCC  ,epsilon = 0.0031163433333176727, years start at= 5    2020.0
Name: year, dtype: float64
GFDL  ,epsilon = 0.0005098100000395789, years start at= 47    2062.0
Name: year, dtype: float64
MPI  ,epsilon = 0.0005014666667193524, years start at= 55    2070.0
Name: year, dtype: float64
--------------------------------------------
warming level = 2
CMCC  ,epsilon = 0.014842613333371446, years start at= 7    2022.0
Name: year, dtype: float64
TaiESM1  ,epsilon = 0.09314801000004991, years start at= 0    2015.0
Name: year, dtype: float64
--------------------------------------------
warming level = 3
CMCC  ,epsilon = 0.010767350000037368, years start at= 50    2065.0
Name: year, dtype: float64
TaiESM1  ,epsilon = 0.012823793333325284, years start at= 23    2038.0
Name: year, dtype: float64
--------------------------------------------
warming level = 4
--------------------------------------------
**********_ssp245
warming level = 1.5
BCC  ,epsilon = 0.0094